#### Testing combinations(x,x) with starts and stops as indices

This is intended to be used with oamap version ( I know we don't intend to finally use it, but let's go with it for now).

##### Idea for calculation

Using the kernel structure used in test_combinations, we can  generate combinations arrays for every event. 

For that, each event is assigned a block ( i.e. each block in the grid serves as an event), and the combinations is calculated for each block using the `starts[block_index]` and event length.
This is appended to `left` and `right` arrays. 

In [8]:
import pycuda.autoinit
import pycuda
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
from pycuda.compiler import *
import numpy

In [9]:
# The generation step. Let's form a random integer array, from which we will form starts 
# and stops arrays
base_len = 32
base_arr = numpy.random.randint(6, size=base_len)

In [10]:
cumul_arr = numpy.zeros(base_len+1)
cumul_arr[0] = 0
cumul_arr[1:] = numpy.cumsum(base_arr)

In [11]:
start = cumul_arr[:-2]
stop = cumul_arr[1:-1]

In [14]:
lengths = stop-start
pairs_lengths = numpy.zeros(len(lengths)+1, dtype=numpy.int32)
pairs_lengths[1:] = numpy.cumsum(lengths*lengths)
start = start.astype(numpy.int32)
lengths_arr = numpy.array([base_len]).astype(numpy.int32)
lengths = lengths.astype(numpy.int32)

In [17]:
# Now let's form the cuda function

mod = SourceModule('''
__global__ void comb_events(int* left,int* right,int* start,int* length,int* lengths,int* pairs_lengths)
{
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    int j =  blockIdx.y*blockDim.y+threadIdx.y;
    int k =  blockIdx.z*blockDim.z+threadIdx.z;
    if (i <length[0] && j< lengths[i] && k<lengths[i])
    {
        left[pairs_lengths[i] + j*lengths[i] + k] = j + start[i];
        right[pairs_lengths[i] + j*lengths[i] + k] = k + start[i];
    }
}
''')

In [18]:
func = mod.get_function("comb_events")

In [19]:
left = numpy.zeros(pairs_lengths[-1]).astype(numpy.int32)
right = numpy.zeros(pairs_lengths[-1]).astype(numpy.int32)

In [20]:
func(cuda.InOut(left), cuda.InOut(right), cuda.In(start), cuda.In(lengths_arr),
    cuda.In(lengths), cuda.In(pairs_lengths), block=(1, 8, 8), grid=(base_len, 1))

In [21]:
left

array([ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,
        3,  3,  3,  4,  4,  4,  4,  4,  5,  5,  6,  6,  7,  7,  7,  7,  8,
        8,  8,  8,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 12, 12, 13, 13,
       13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16,
       16, 17, 17, 17, 17, 17, 18, 18, 19, 19, 20, 20, 20, 21, 21, 21, 22,
       22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 27, 27, 27, 28, 28,
       28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 31,
       31, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 35,
       35, 35, 35, 35, 36, 36, 36, 36, 36, 37, 37, 38, 38, 39, 39, 39, 39,
       39, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 43,
       43, 43, 43, 43, 44, 44, 45, 45, 46, 46, 46, 46, 47, 47, 47, 47, 48,
       48, 48, 48, 49, 49, 49, 49, 50, 50, 50, 50, 51, 51, 51, 51, 52, 52,
       52, 52, 53, 53, 53, 53, 54, 54, 54, 55, 55, 55, 56, 56, 56, 57, 57,
       58, 58, 59, 60, 60

In [22]:
right

array([ 0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,
        2,  3,  4,  0,  1,  2,  3,  4,  5,  6,  5,  6,  7,  8,  9, 10,  7,
        8,  9, 10,  7,  8,  9, 10,  7,  8,  9, 10, 11, 12, 11, 12, 13, 14,
       15, 16, 17, 13, 14, 15, 16, 17, 13, 14, 15, 16, 17, 13, 14, 15, 16,
       17, 13, 14, 15, 16, 17, 18, 19, 18, 19, 20, 21, 22, 20, 21, 22, 20,
       21, 22, 23, 24, 23, 24, 25, 26, 25, 26, 27, 28, 29, 30, 31, 27, 28,
       29, 30, 31, 27, 28, 29, 30, 31, 27, 28, 29, 30, 31, 27, 28, 29, 30,
       31, 32, 33, 34, 35, 36, 32, 33, 34, 35, 36, 32, 33, 34, 35, 36, 32,
       33, 34, 35, 36, 32, 33, 34, 35, 36, 37, 38, 37, 38, 39, 40, 41, 42,
       43, 39, 40, 41, 42, 43, 39, 40, 41, 42, 43, 39, 40, 41, 42, 43, 39,
       40, 41, 42, 43, 44, 45, 44, 45, 46, 47, 48, 49, 46, 47, 48, 49, 46,
       47, 48, 49, 46, 47, 48, 49, 50, 51, 52, 53, 50, 51, 52, 53, 50, 51,
       52, 53, 50, 51, 52, 53, 54, 55, 56, 54, 55, 56, 54, 55, 56, 57, 58,
       57, 58, 59, 60, 61

In [42]:
# Print indices for each event
for i in range(5):
    print("Event: {}".format(i))
    print ("Left Index: {}".format(left[pairs_lengths[i]:pairs_lengths[i+1]]))
    print("Right Index: {}".format(right[pairs_lengths[i]:pairs_lengths[i+1]]))

Event: 0
Left Index: [0 0 0 0 0 1 1 1 1 1 2 2 2 2 2 3 3 3 3 3 4 4 4 4 4]
Right Index: [0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4]
Event: 1
Left Index: [5 5 6 6]
Right Index: [5 6 5 6]
Event: 2
Left Index: [ 7  7  7  7  8  8  8  8  9  9  9  9 10 10 10 10]
Right Index: [ 7  8  9 10  7  8  9 10  7  8  9 10  7  8  9 10]
Event: 3
Left Index: [11 11 12 12]
Right Index: [11 12 11 12]
Event: 4
Left Index: [13 13 13 13 13 14 14 14 14 14 15 15 15 15 15 16 16 16 16 16 17 17 17 17 17]
Right Index: [13 14 15 16 17 13 14 15 16 17 13 14 15 16 17 13 14 15 16 17 13 14 15 16 17]
